In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
%cd /content/drive/MyDrive/강의자료/머신러닝기초/3일차_회귀

[WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/drive/MyDrive/강의자료/머신러닝기초/3일차_회귀'
C:\Users\BIG2-003-007\Desktop\야


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 데이터 불러오기 : 자전거 대여량 수요 예측

In [3]:
df = pd.read_csv('./datasets/bike_train.csv')
pd.set_option('display.max_columns', len(df.columns))
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


주요 컬럼 설명
- datetime: hourly date + timestamp
- season: 1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울
- holiday: 1 = 토, 일요일의 주말을 제외한 국경일 등의 휴일, 0 = 휴일이 아닌 날
- workingday: 1 = 토, 일요일의 주말 및 휴일이 아닌 주중, 0 = 주말 및 휴일
- weather: 1 = 맑음, 약간 구름 낀 흐림 2 = 안개, 안개 + 흐림 3 = 가벼운 눈, 가벼운 비 + 천둥 4 = 심한 눈/비, 천둥/번개
- temp: 온도(섭씨)
- atemp: 체감온도(섭씨)
- humidity: 상대습도
- windspeed: 풍속
- casual: 사전에 등록되지 않는 사용자가 대여한 횟수
- registered: 사전에 등록된 사용자가 대여한 횟수
- count: 대여 횟수

자전거 대여 횟수(Y) 예측을 위해 X를 만들어 모델에 적용해보세요!

In [4]:
# TODO : 데이터 전처리 자유롭게

자전거 대여에 영향을 끼치는 컬럼만 가져오기

In [5]:
df.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [6]:
df = df[['datetime','season','holiday','workingday','weather','temp','atemp','humidity','windspeed','count']]

In [7]:
df.info() #결측시 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   count       10886 non-null  int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 850.6+ KB


In [8]:
df['datetime'] = df.datetime.apply(pd.to_datetime)
df['hour'] = df.datetime.apply(lambda x: x.hour) #데이타임 시간컬럼 추가

In [9]:
df['datetime'] = df.datetime.apply(pd.to_datetime)
df['month'] = df.datetime.apply(lambda x: x.month) #데이타임 월별 컬럼 추가

In [10]:
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,month
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,16,0,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,40,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,32,2,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,13,3,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,336,19,12
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,241,20,12
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,168,21,12
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,129,22,12


In [11]:
df = df.drop('datetime', axis=1) #데이타임 컬럼 날리기

In [12]:
df = df.drop_duplicates() #중복값 날리기

In [13]:
df

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,month
0,1,0,0,1,9.84,14.395,81,0.0000,16,0,1
1,1,0,0,1,9.02,13.635,80,0.0000,40,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,32,2,1
3,1,0,0,1,9.84,14.395,75,0.0000,13,3,1
4,1,0,0,1,9.84,14.395,75,0.0000,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,336,19,12
10882,4,0,1,1,14.76,17.425,57,15.0013,241,20,12
10883,4,0,1,1,13.94,15.910,61,15.0013,168,21,12
10884,4,0,1,1,13.94,17.425,61,6.0032,129,22,12


In [14]:
df.info() #문자형 데이터가 있는지 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10881 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10881 non-null  int64  
 1   holiday     10881 non-null  int64  
 2   workingday  10881 non-null  int64  
 3   weather     10881 non-null  int64  
 4   temp        10881 non-null  float64
 5   atemp       10881 non-null  float64
 6   humidity    10881 non-null  int64  
 7   windspeed   10881 non-null  float64
 8   count       10881 non-null  int64  
 9   hour        10881 non-null  int64  
 10  month       10881 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 1020.1 KB


In [15]:
X = df[['season','holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'hour', 'month']].to_numpy()
Y = df['count'].to_numpy()

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [17]:
y_train

array([279,   2, 283, ..., 336, 310, 383], dtype=int64)

In [18]:
y_test # 학습 : 1,2,3 테스트 : 1,2,3,4

array([107, 203,  84, ...,  12, 199, 218], dtype=int64)

### 모델 적용 / 평가

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 사이킷런의 mean_square_error() 를 이용하여 RMSE 계산
def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

# MSE, RMSE, RMSLE 를 모두 계산
def evaluate_regr(y,pred):
    rmse_val = rmse(y,pred)
    # MAE 는 scikit learn의 mean_absolute_error() 로 계산
    mae_val = mean_absolute_error(y,pred)
    print('RMSE: {0:.3F}, MAE: {1:.3F}'.format(rmse_val, mae_val))

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = df['count']
X = df.drop(['count'],axis=1,inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr_reg = LinearRegression()

In [21]:
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

evaluate_regr(y_test ,pred)

RMSE: 146.242, MAE: 107.199


In [24]:
pd.DataFrame(data=[y_test.to_numpy(), pred])

,0,1,2,3,4,5,...,2171,2172,2173,2174,2175,2176
0,107.000000,203.000000,84.000000,89.00000,131.000000,5.000000,...,722.000000,166.000000,118.00000,12.000000,199.00000,218.000000
1,116.578031,247.731869,10.298263,-12.12031,387.199407,85.312178,...,394.429512,-7.342698,242.14048,190.513859,100.16454,272.300402


In [ ]:
X

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, confusion_matrix

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, max_depth=8)

rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)

In [26]:
pred

array([114, 303,  64, ...,  10,  63, 193], dtype=int64)

In [32]:
y_test

4255     107
10472    203
144       84
5690      89
8270     131
        ... 
6124     166
4627     118
3655      12
5165     199
9540     218
Name: count, Length: 2177, dtype: int64

In [35]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred, average = 'micro')

(0.022048690858980247, 0.022048690858980247, 0.022048690858980247, None)